# Ejemplo de Voice Bot informativo implementado al 100% en Python

### Es un ejemplo sencillo de voicebot que emplea librerías para el reconocimiento y la síntesis de voz.

### Te recomiendo que si no estás familiarizado con estas soluciones, primero revises el ejemplo de chatbot sencillo. Este notebook está basado en él

#### El voicebot informa a los usuarios acerca de las normas de un crucero. Es un ejemplo básico, pero que bien sirve de ejemplo de uso de lematización y búsqueda de coincidencias entre las preguntas de usuario y las diferentes respuestas posibles mediante el modelo "cosine_similarity"

#### Resumen técnico.

##### 1.- En una variable de texto se almacena el corpus (diferentes respuestas posibles al usuario).
##### 2.- Cuando el usuario plantea una pregunta, se agrega -temporalmente- al final de la lista de respuestas. A todo este contenido se le eliminan signos de puntuación, se tokeniza, lematiza y se extraen sus caracterísaticas -mediante TfidfVectorizer de sklearn-. A partir de ellas y empleando un modelo del tipo "cosine_similarity" se buscan las respuestas más coincidentes con la pregunta del usuario, se elige la que mayor grado de coincidentcia muestra y se responde con ella.
##### 3.- Adicionalmente se ha incluido un pequeño módulo de saludo inicial y de despedida, que aleatoriamente elige una respuesta entre varias posibles.


In [1]:
# Importación de librerías
import nltk
import numpy as np
import random
import string

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords

#nltk.download('punkt') # Instalar módulo punkt si no está ya instalado (solo ejecutar la primera vez)
#nltk.download('wordnet') # Instalar módulo wordnet si no está ya instalado (solo ejecutar la primera vez)

import speech_recognition as sr # librería para reconocimiento de voz
import pyttsx3 # librería para síntesis de voz



#### 1 Carga del corpus

In [2]:
# Debe modificarse para que indique exactamente la ubicación del archivo que contiene el corpus. 
# Puede sustiruirse por un corpus sobrecualquier otra temática
f=open(r'C:\Corpus_crucero.txt','r',errors = 'ignore')
raw=f.read()
#print(raw)  #Si deseas ver el corpus, descomenta esta línea

#### 2 Definición de funciones y variables de apoyo

In [3]:
raw=raw.lower() # Convertimos todo el texto a minúsculas, para evitar deficiencias en la extracción de características

sent_tokens = nltk.sent_tokenize(raw) # Convierte el corpus a una lista de sentencias
word_tokens = nltk.word_tokenize(raw) # Convierte el corpus a una lista de palabras

lemmer = nltk.stem.WordNetLemmatizer() # Instanciamos el lematizador, con el que convertir las palabras  a sus raíces contextuales

#LemTokens es una función que lematiza todos los tokens que se le pasan como parámetro
def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]

# remove_punct es un diccionario del tipo (0signo de puntuación', None), que se emplea en la función
# LemNormalize para sustituir los signos de puntuación por "nada" es decir, eliminarlos.
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)

# Dado un texto como parámetro, elimina los signos de puntuación, lo convierte a minúsculas,
# lo tokeniza -por palabras- y finalmente lo lematiza
def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))



#### 3 Preprocesamiento del texto y evaluación de la similitud entre el mensaje de usuario y las respuestas definidas en el corpus

In [4]:
# Función para determinar la similitud del texto insertado y el corpus
def response(user_response):
    
    robo_response=''
    sent_tokens.append(user_response) # Añade al final del corpus la respuesta de usuario. Se hace esto para que al usasr cosine_similarity las shapes de pregunta y respuestas coincidan y no de error
    
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words=stopwords.words('spanish'))
    
    caract_textos = TfidfVec.fit_transform(sent_tokens)
    
    # Ahora vamos a evaluar la similitud y devolver la respuesta adecuada a partir del corpus
    
    # vals es un vector con los grados de coincidencia entre la pregunta y los textos.
    # la última línea de los textos es una pregunta, por lo que en ese caso será siempre 1, nos interesa el inmediato siguiente
    vals = cosine_similarity(caract_textos[-1], caract_textos) 
    
    # argsort ordena en orden creciente el vector, pero devuelve los índices originales, no los valores, por lo que
    # en idx tendremos ekl índice del término que más se ajusta como respuesta.
    idx=vals.argsort()[0][-2] # Se ordena el vector de coincidencias (ascendente) y se toma el índice del penúltimo -el último es la pregunta de usuario-, que es el de mayor coincidencia
    
    flat = vals.flatten()  # eliminamos una de las dimensiones convirtiendo vals en vector
    flat.sort()
    nivel_coincidencia = flat[-2] # obtenemos el nivel de coincidencia para saber si hay una respuesta válida
    
    if(nivel_coincidencia==0):
        robo_response=robo_response+"Lo siento, no te he entendido. Si no puedo responder a lo que busca póngase en contacto con atención al cliente en el 902.902.902 (llamada local)"
        return robo_response
    else:
        robo_response = robo_response+sent_tokens[idx]
        return robo_response

#### 4 Definición de funcionalidades de saludo y despedida

In [5]:
saludo_inputs = ("hola", "buenas", "saludos", "qué tal", "hey","buenos dias",)
saludo_outputs = ["Hola", "Hola, ¿Qué tal?", "Hola, ¿Cómo te puedo ayudar?", "Hola, encantado de hablar contigo"]


despedidas = ["Nos vemos", "Hasta pronto", "Nos vemos en otro rato", "Chao", "Bye", "Un placer, espero haberte ayudado"]

def saludos(sentence):
    for word in sentence.split():
        if word.lower() in saludo_inputs:
            return random.choice(saludo_outputs)
        
def despedida():
    return random.choice(despedidas)

##### 5 Configuración del reconocedor de voz

In [6]:
# Se instancia el reconocedor en 'r'
r = sr.Recognizer()

# Si queremos interpretar el audio desde un fichero
#audio_file = sr.AudioFile('audio_ejemplo.wav') # El fichero debe estar en la ruta del intérprete de Python

# si en el fichero de audio hay mucho y solo nos interesa una parte...
#with audio_file as source:
#    r.adjust_for_ambient_noise(source) # Calibración si el fichero tiene ruido de fondo
#    audio = r.record(source) # si solo se quiere una porción del audio: audio = r.record(source, offset=4, duration=3)

# Si audio desde micrófono
mic = sr.Microphone()

# Si se necesita seleccionar un micrófono que no sea el de por defecto
#sr.Microphone.list_microphone_names()
#mic = sr.Microphone(device_index=1) #1 es el micro por defecto,  6 sería para el micrófono 6 del listado

# Definición de la función que va a encargarse de reconocer la voz del usuario y convertirla a texto
def reconocer_voz():
    retorno = ''
    while (retorno==''):
        try:
            with mic as source:
                r.adjust_for_ambient_noise(source) # Calibración si el audio tiene ruido de fondo
                audio = r.listen(source)
            retorno = r.recognize_google(audio, language='es-ES') # Definimos la API a utilizar como método de reconocimiento, la única que no necesita login es la de Google (debemos estar conectados a internet)
        except:
            retorno = ''
    return retorno


##### 6 Configuración del sistema de síntesis de voz

In [7]:
# instancia del sistema de síntesis de voz
engine = pyttsx3.init()


engine.setProperty('rate', 180)    # Aquí puedes seleccionar la velocidad de la voz
engine.setProperty('voice', 'spanish') # configuración

#definición de la función que reproduce una respuesta del voice bot
def habla(texto):
    engine.say(texto)
    engine.runAndWait()
    

#### 7 Bucle conversacional

In [8]:
flag=True
#print("ROBOT: Mi nombre es ROBOT. Contestaré a tus preguntas acerca de sus vacaciones en el crucero. Si quieres salir, solamente di 'salir' ")
habla("Mi nombre es ROBOT. Contestaré a tus preguntas acerca de sus vacaciones en el crucero. Si quieres salir, solamente diga 'salir' ")
while(flag==True):
    #user_response = input()
    user_response = reconocer_voz()
    #print(user_response)
    user_response = user_response.lower() #Convertimos a minúscula
    
    if(user_response!='salir'):
        
        if(user_response=='gracias' or user_response=='muchas gracias'): #Se podría haber definido otra función de coincidencia manual
            flag=True
            habla("No hay de qué")
            
        else:
            if(saludos(user_response)!=None): #Si la palabra insertada por el usuario es un saludo (Coincidencias manuales definidas previamente)
                habla(saludos(user_response))
                
            else: #Si la palabra insertada no es un saludo --> CORPUS
                #print("ROBOT: ",end="") 
                habla(response(user_response))
                sent_tokens.remove(user_response) # para eliminar del corpus la respuesta del usuario y volver a evaluar con el CORPUS limpio
    else:
        flag=False
        habla(despedida())